In [0]:
!pip install nltk
#nltk is required for our projcet

In [0]:
#importing required libraries
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import col
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline


In [0]:
#Loading data into a daframe
df = spark.read.csv("/FileStore/tables/fake_job_postings.csv",header = True,inferSchema = True)

In [0]:
#displaying the loaded data
df.show()

+------+--------------------+--------------------+----------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+
job_id| title| location|department| salary_range| company_profile| description| requirements| benefits| telecommuting|has_company_logo|has_questions|employment_type|required_experience| required_education| industry| function|fraudulent|
+------+--------------------+--------------------+----------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+
 1| Marketing Intern| US, NY, New York| Marketing| null|We're Food52, and...|Food52, a fast-gr...|Experience with c...| null| 0| 1| 0| Other| Internship| null| null| Marketing| 0|
 2|Customer Service ...| NZ, , Auckland| Success| null|90 Seconds, the w...|Organised - Focus...|What we expect fr...|What you will get...| 0| 1| 0| Full-time| Not Applicable| null|Marketing and Adv...| Customer Service| 0|
 3|Commissioning Mac...| US, IA, Wever| null| null|Valor Services pr...|Our client, locat...|Implement pre-com...| null| 0| 1| 0| null| null| null| null| null| 0|
 4|Account Executive...| US, DC, Washington| Sales| null|Our passion for i...|THE COMPANY: ESRI...|EDUCATION: Bachel...|Our culture is an...| 0| 1| 0| Full-time| Mid-Senior level| Bachelor's Degree| Computer Software| Sales| 0|
 5| Bill Review Manager| US, FL, Fort Worth| null| null|SpotSource Soluti...|JOB TITLE: Itemiz...|QUALIFICATIONS:RN...|Full Benefits Off...| 0| 1| 1| Full-time| Mid-Senior level| Bachelor's Degree|Hospital & Health...|Health Care Provider| 0|
 6| Accounting Clerk| US, MD, | null| null| null|Job OverviewApex ...| null| null| 0| 0| 0| null| null| null| null| null| 0|
 7|Head of Content (...| DE, BE, Berlin|ANDROIDPIT| 20000-28000|Founded in 2009, ...|Your Responsibili...|Your Know-How:   ...|Your Benefits: Be...| 0| 1| 1| Full-time| Mid-Senior level| Master's Degree| Online Media| Management| 0|
 8|Lead Guest Servic...|US, CA, San Franc...| null| null|Airenvy’s mission...|Who is Airenvy?He...|Experience with C...|Competitive Pay. ...| 0| 1| 1| null| null| null| null| null| 0|
 9| HP BSM SME| US, FL, Pensacola| null| null|Solutions3 is a w...|Implementation/Co...|MUST BE A US CITI...| null| 0| 1| 1| Full-time| Associate| null|Information Techn...| null| 0|
 10|Customer Service ...| US, AZ, Phoenix| null| null|Novitex Enterpris...|The Customer Serv...|Minimum Requireme...| null| 0| 1| 0| Part-time| Entry level|High School or eq...| Financial Services| Customer Service| 0|
 11|ASP.net Developer...| US, NJ, Jersey City| null|100000-120000| null|Position : #URL_8...|Position : #URL_8...|Benefits - FullBo...| 0| 0| 0| Full-time| Mid-Senior level| Bachelor's Degree|Information Techn...|Information Techn...| 0|
 12|Talent Sourcer (6...| GB, LND, London| HR| null|Want to build a 2...|TransferWise is t...|We’re looking for...|You will join one...| 0| 1| 0| null| null| null| null| null| 0|
 13|Applications Deve...| US, CT, Stamford| null| null|Novitex Enterpris...|The Applications ...|Requirements:4 – ...| null| 0| 1| 0| Full-time| Associate| Bachelor's Degree|Management Consul...|Information Techn...| 0|
 14| Installers| US, FL, Orlando| null| null|Growing event pro...|Event Industry In...|Valid driver's li...| null| 0| 1| 1| Full-time| Not Applicable| Unspecified| Events Services| Other| 0|
 15|Account Executive...| AU, NSW, Sydney| Sales| null|Adthena is the UK...|Are you intereste...|You’ll need to be...|In return we'll p...| 0| 1| 0| Full-time| Associate| Bachelor's Degree| Internet| Sales| 0|
 16|VP of Sales - Vau...| SG, 01, Singapore| Sales|120000-150000|Jungle Ventures i...|About Vault D

In [0]:
#displaying the columns in a dataframe
df.columns

Out[5]: ['job_id',
 'title',
 'location',
 'department',
 'salary_range',
 'company_profile',
 'description',
 'requirements',
 'benefits',
 'telecommuting',
 'has_company_logo',
 'has_questions',
 'employment_type',
 'required_experience',
 'required_education',
 'industry',
 'function',
 'fraudulent']

In [0]:
#counting null values in each column
from pyspark.sql.functions import isnan
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------+-----+--------+----------+------------+---------------+-----------+------------+--------+-------------+----------------+-------------+---------------+-------------------+------------------+--------+--------+----------+
job_id|title|location|department|salary_range|company_profile|description|requirements|benefits|telecommuting|has_company_logo|has_questions|employment_type|required_experience|required_education|industry|function|fraudulent|
+------+-----+--------+----------+------------+---------------+-----------+------------+--------+-------------+----------------+-------------+---------------+-------------------+------------------+--------+--------+----------+
 0| 0| 346| 11547| 15011| 3308| 1| 2573| 6966| 89| 29| 30| 3292| 6723| 7748| 4831| 6317| 176|
+------+-----+--------+----------+------------+---------------+-----------+------------+--------+-------------+----------------+-------------+---------------+-------------------+------------------+--------+--------+----------+

In [0]:
#dropping unnecessary columns
df1 = df.drop("salary_range","job_id")

In [0]:
#printing schema of dataframe
df1.printSchema()

root
-- title: string (nullable = true)
-- location: string (nullable = true)
-- department: string (nullable = true)
-- company_profile: string (nullable = true)
-- description: string (nullable = true)
-- requirements: string (nullable = true)
-- benefits: string (nullable = true)
-- telecommuting: string (nullable = true)
-- has_company_logo: string (nullable = true)
-- has_questions: string (nullable = true)
-- employment_type: string (nullable = true)
-- required_experience: string (nullable = true)
-- required_education: string (nullable = true)
-- industry: string (nullable = true)
-- function: string (nullable = true)
-- fraudulent: string (nullable = true)

In [0]:
#replacing null values with space
df1 = df1.fillna("")

In [0]:
df1.show()

+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+
 title| location|department| company_profile| description| requirements| benefits| telecommuting|has_company_logo|has_questions|employment_type|required_experience| required_education| industry| function|fraudulent|
+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+---------------+-------------------+--------------------+--------------------+--------------------+----------+
 Marketing Intern| US, NY, New York| Marketing|We're Food52, and...|Food52, a fast-gr...|Experience with c...| | 0| 1| 0| Other| Internship| | | Marketing| 0|
Customer Service ...| NZ, , Auckland| Success|90 Seconds, the w...|Organised - Focus...|What we expect fr...|What you will get...| 0| 1| 0| Full-time| Not Applicable| |Marketing and Adv...| Customer Service| 0|
Commissioning Mac...| US, IA, Wever| |Valor Services pr...|Our client, locat...|Implement pre-com...| | 0| 1| 0| | | | | | 0|
Account Executive...| US, DC, Washington| Sales|Our passion for i...|THE COMPANY: ESRI...|EDUCATION: Bachel...|Our culture is an...| 0| 1| 0| Full-time| Mid-Senior level| Bachelor's Degree| Computer Software| Sales| 0|
 Bill Review Manager| US, FL, Fort Worth| |SpotSource Soluti...|JOB TITLE: Itemiz...|QUALIFICATIONS:RN...|Full Benefits Off...| 0| 1| 1| Full-time| Mid-Senior level| Bachelor's Degree|Hospital & Health...|Health Care Provider| 0|
 Accounting Clerk| US, MD, | | |Job OverviewApex ...| | | 0| 0| 0| | | | | | 0|
Head of Content (...| DE, BE, Berlin|ANDROIDPIT|Founded in 2009, ...|Your Responsibili...|Your Know-How:   ...|Your Benefits: Be...| 0| 1| 1| Full-time| Mid-Senior level| Master's Degree| Online Media| Management| 0|
Lead Guest Servic...|US, CA, San Franc...| |Airenvy’s mission...|Who is Airenvy?He...|Experience with C...|Competitive Pay. ...| 0| 1| 1| | | | | | 0|
 HP BSM SME| US, FL, Pensacola| |Solutions3 is a w...|Implementation/Co...|MUST BE A US CITI...| | 0| 1| 1| Full-time| Associate| |Information Techn...| | 0|
Customer Service ...| US, AZ, Phoenix| |Novitex Enterpris...|The Customer Serv...|Minimum Requireme...| | 0| 1| 0| Part-time| Entry level|High School or eq...| Financial Services| Customer Service| 0|
ASP.net Developer...| US, NJ, Jersey City| | |Position : #URL_8...|Position : #URL_8...|Benefits - FullBo...| 0| 0| 0| Full-time| Mid-Senior level| Bachelor's Degree|Information Techn...|Information Techn...| 0|
Talent Sourcer (6...| GB, LND, London| HR|Want to build a 2...|TransferWise is t...|We’re looking for...|You will join one...| 0| 1| 0| | | | | | 0|
Applications Deve...| US, CT, Stamford| |Novitex Enterpris...|The Applications ...|Requirements:4 – ...| | 0| 1| 0| Full-time| Associate| Bachelor's Degree|Management Consul...|Information Techn...| 0|
 Installers| US, FL, Orlando| |Growing event pro...|Event Industry In...|Valid driver's li...| | 0| 1| 1| Full-time| Not Applicable| Unspecified| Events Services| Other| 0|
Account Executive...| AU, NSW, Sydney| Sales|Adthena is the UK...|Are you intereste...|You’ll need to be...|In return we'll p...| 0| 1| 0| Full-time| Associate| Bachelor's Degree| Internet| Sales| 0|
VP of Sales - Vau...| SG, 01, Singapore| Sales|Jungle Ventures i...|About Vault Drago...|Key Superpowers3-...|"Basic: SGD 120,0...| the way you want...| 0| 1| 1| Full-time| Executive| Bachelor's Degree| Facilities Services| Sales|
 Hands-On QA Leader |IL, , Tel Aviv, I...| R&D|At HoneyBook we’r...|We are looking fo...|Previous experien...| | 0| 1| 0| Full-time| Mid-Senior level| | Internet| Engineering| 0|
Southend-on-Sea T...|GB, SOS, Southend...| |Established on th...|Government fundin..

In [0]:
#merging columns into a text column
df1 = df1.withColumn('text', 
                    concat(col('title') ,lit(' ') , col('location') ,lit(' ') ,col('department'), lit(' '),col('company_profile'), lit(' '), col('description'),lit(' ') ,col('requirements'),lit(' '),col('benefits'),lit(' '),col('employment_type'),
                           lit(' '),col('required_education'),lit(' '),col('industry'),lit(' '),col('function')))

In [0]:
#displaying the text column
df1.select('text').show(1)

+--------------------+
 text|
+--------------------+
Marketing Intern ...|
+--------------------+
only showing top 1 row

In [0]:
#removing the columns
df1 = df1.drop('title','location','department','company_profile','description','requirements','benefits','employment_type','required_experience'
       ,'required_education','industry','function')

In [0]:
#creating null values columns to find out the null values in each column
df2 = df1.select(
  "telecommuting",col("telecommuting").cast("int").isNotNull().alias("tele_Value"),
  "has_company_logo",col("has_company_logo").cast("int").isNotNull().alias("logo_Value"),
  "has_questions",col("has_questions").cast("int").isNotNull().alias("question_Value"),
  "fraudulent",col("fraudulent").cast("int").isNotNull().alias("fradulent_Value"),
  "text")

In [0]:
df2.printSchema()

root
-- telecommuting: string (nullable = false)
-- tele_Value: boolean (nullable = false)
-- has_company_logo: string (nullable = false)
-- logo_Value: boolean (nullable = false)
-- has_questions: string (nullable = false)
-- question_Value: boolean (nullable = false)
-- fraudulent: string (nullable = false)
-- fradulent_Value: boolean (nullable = false)
-- text: string (nullable = false)

In [0]:
#selecting only which have true value
df3 = df2.filter(df2['tele_Value'] == 'true').filter(df2['logo_Value'] == 'true').filter(df2['question_Value'] == 'true').filter(df2['fradulent_Value'] == 'true')

In [0]:
df3.printSchema()

root
-- telecommuting: string (nullable = false)
-- tele_Value: boolean (nullable = false)
-- has_company_logo: string (nullable = false)
-- logo_Value: boolean (nullable = false)
-- has_questions: string (nullable = false)
-- question_Value: boolean (nullable = false)
-- fraudulent: string (nullable = false)
-- fradulent_Value: boolean (nullable = false)
-- text: string (nullable = false)

In [0]:
#finding if there are any null values in the given column
df3.filter(df3['fradulent_Value'] == 'false').count()

Out[18]: 0

In [0]:
df3.count()

Out[19]: 16844

In [0]:
#dropping rows if there are any null values
df3=df3.dropna(how='any')
df3.count()

Out[21]: 16844

In [0]:
df3.printSchema()

root
-- telecommuting: string (nullable = false)
-- tele_Value: boolean (nullable = false)
-- has_company_logo: string (nullable = false)
-- logo_Value: boolean (nullable = false)
-- has_questions: string (nullable = false)
-- question_Value: boolean (nullable = false)
-- fraudulent: string (nullable = false)
-- fradulent_Value: boolean (nullable = false)
-- text: string (nullable = false)

In [0]:
#dropping newly created columns
df3 = df3.drop('tele_Value','logo_Value','question_Value','fradulent_Value')

In [0]:
#casting output column to integer
df3 = df3.withColumn('fraudulent',df3.fraudulent.cast('integer'))

In [0]:
df3.printSchema()

root
-- telecommuting: string (nullable = false)
-- has_company_logo: string (nullable = false)
-- has_questions: string (nullable = false)
-- fraudulent: integer (nullable = true)
-- text: string (nullable = false)

##Data Visualization

In [0]:
#%sql
#select telecommuting,count(telecommuting) as telecommuting_count from jobdescription_csv group by telecommuting
display(df3.groupby("telecommuting").agg(count("telecommuting").alias("telecommuting_count")))

telecommuting,telecommuting_count
0,16137
1,707


From the above bar plot,we can see there are more number of jobs available for "telecommuting" 0,the count is 16137 and for value 1 only
707 available.

In [0]:
#%sql
#select has_company_logo,count(has_company_logo) as has_company_logo_count from jobdescription_csv group by has_company_logo
display(df3.groupby("has_company_logo").agg(count("has_company_logo").alias("has_company_logo_count")))

has_company_logo,has_company_logo_count
0,3552
1,13292


From the above pi plot,we can see there are more number of jobs available "has_company_logo" 1,the % count is 79 and for value 0 the % count is 21.

In [0]:
#%sql
#select has_questions,count(has_questions) as has_questions_count from jobdescription_csv group by has_questions
display(df3.groupby("has_questions").agg(count("has_questions").alias("has_questions_count")))

has_questions,has_questions_count
0,8673
1,8171


From the above bar plot ,we can see there are almost equal number of jobs available for the variable "has_questions" values 0 and 1.

In [0]:
#%sql
#select fraudulent,count(fraudulent) as fraudulent_count from jobdescription_csv group by fraudulent
display(df3.groupby("fraudulent").agg(count("fraudulent").alias("fraudulent_count")))


fraudulent,fraudulent_count
1,832
0,16012


From the above plot,we can see there are very less number of fake jobs compared to real jobs.

###Text Analytics on the data set

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.sql.types import *

# implementing a df with categorical columns and cleaning the text data
df_clean = df3.select('telecommuting','has_company_logo','has_questions','fraudulent',(lower(regexp_replace('text', "[^a-zA-Z\\s]", "")).alias('text')))

# using tokenizer on text data
tokenizer = Tokenizer(inputCol='text', outputCol='words_token')
df_words_token = tokenizer.transform(df_clean).select('telecommuting','has_company_logo','has_questions','fraudulent','words_token')

# stop words removal
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
df_words_no_stopw = remover.transform(df_words_token).select('telecommuting','has_company_logo','has_questions','fraudulent','words_clean')

# using stemmer to stem the words
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
df_stemmed = df_words_no_stopw.withColumn("words_stemmed", stemmer_udf("words_clean")).select('telecommuting','has_company_logo','has_questions','fraudulent','words_stemmed')

# selecting only words with length has greater than 3
filter_length_udf = udf(lambda row: [x for x in row if len(x) >= 3], ArrayType(StringType()))
df_final_words = df_stemmed.withColumn('words', filter_length_udf(col('words_stemmed')))

In [0]:
df_final_words.count()

Out[31]: 16844

In [0]:
display(df_final_words)

telecommuting has_company_logo has_questions fraudulent words_stemmed words 0 1 0 0 List(market, intern, us, ny, new, york, market, food, weve, creat, groundbreak, awardwin, cook, site, support, connect, celebr, home, cook, give, everyth, need, one, placew, top, editori, busi, engin, team, focus, use, technolog, find, new, better, way, connect, peopl, around, specif, food, interest, offer, superb, high, curat, inform, food, cook, attract, talent, home, cook, contributor, countri, also, publish, wellknown, profession, like, mario, batali, gwyneth, paltrow, danni, meyer, partnership, whole, food, market, random, housefood, name, best, food, websit, jame, beard, foundat, iacp, featur, new, york, time, npr, pando, daili, techcrunch, today, showwer, locat, chelsea, new, york, citi, food, fastgrow, jame, beard, awardwin, onlin, food, communiti, crowdsourc, curat, recip, hub, current, interview, full, parttim, unpaid, intern, work, small, team, editor, execut, develop, new, york, citi, headquartersreproduc, andor, repackag, exist, food, content, number, partner, site, huffington, post, yahoo, buzzfe, various, content, manag, systemsresearch, blog, websit, provis, food, affili, programassist, daytoday, affili, program, support, screen, affili, assist, affili, inquiriessupport, pr, amp, event, neededhelp, offic, administr, work, file, mail, prepar, meetingswork, develop, document, bug, suggest, improv, sitesupport, market, execut, staff, experi, content, manag, system, major, plus, blog, countsfamiliar, food, editori, voic, aestheticlov, food, appreci, import, home, cook, cook, seasonsmeticul, editor, perfectionist, obsess, attent, detail, madden, typo, broken, link, delight, find, fix, themcheer, pressureexcel, communic, skillsa, multitask, juggler, respons, big, smallinterest, engag, social, media, like, twitter, facebook, pinterestlov, problemsolv, collabor, drive, food, forwardthink, big, pictur, pitch, nitti, gritti, run, small, compani, dish, shop, administr, supportcomfort, realiti, work, startup, call, even, weekend, work, long, hour, , , , market) List(market, intern, new, york, market, food, weve, creat, groundbreak, awardwin, cook, site, support, connect, celebr, home, cook, give, everyth, need, one, placew, top, editori, busi, engin, team, focus, use, technolog, find, new, better, way, connect, peopl, around, specif, food, interest, offer, superb, high, curat, inform, food, cook, attract, talent, home, cook, contributor, countri, also, publish, wellknown, profession, like, mario, batali, gwyneth, paltrow, danni, meyer, partnership, whole, food, market, random, housefood, name, best, food, websit, jame, beard, foundat, iacp, featur, new, york, time, npr, pando, daili, techcrunch, today, showwer, locat, chelsea, new, york, citi, food, fastgrow, jame, beard, awardwin, onlin, food, communiti, crowdsourc, curat, recip, hub, current, interview, full, parttim, unpaid, intern, work, small, team, editor, execut, develop, new, york, citi, headquartersreproduc, andor, repackag, exist, food, content, number, partner, site, huffington, post, yahoo, buzzfe, various, content, manag, systemsresearch, blog, websit, provis, food, affili, programassist, daytoday, affili, program, support, screen, affili, assist, affili, inquiriessupport, amp, event, neededhelp, offic, administr, work, file, mail, prepar, meetingswork, develop, document, bug, suggest, improv, sitesupport, market, execut, staff, experi, content, manag, system, major, plus, blog, countsfamiliar, food, editori, voic, aestheticlov, food, appreci, import, home, cook, cook, seasonsmeticul, editor, perfectionist, obsess, attent, detail, madden, typo, broken, link, delight, find, fix, themcheer, pressureexcel, communic, skillsa, multitask, juggler, respons, big, smallinterest, engag, social, media, like, twitter, facebook, pinterestlov, problemsolv, collabor, drive, food, forwardthink, big, pictur, pitch, nitti, gritti, run, small, compani, dish, shop, administr, supportcomfort, reali

In [0]:
df_final_words.printSchema()

root
-- telecommuting: string (nullable = false)
-- has_company_logo: string (nullable = false)
-- has_questions: string (nullable = false)
-- fraudulent: integer (nullable = true)
-- words_stemmed: array (nullable = true)
 |-- element: string (containsNull = true)
-- words: array (nullable = true)
 |-- element: string (containsNull = true)

In [0]:
df_final = df_final_words.dropna(how='any')

In [0]:
df_final.count()

Out[35]: 16844

In [0]:
#(train_set, test_set) = df_final.randomSplit([0.70, 0.30], seed = 42)
(train_set, test_set) = df_final.randomSplit([0.70, 0.30], seed = 42)

###Logistic Regression model

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

# Generating tf-idf from the text data 

hashingtf = HashingTF(inputCol="words", outputCol='tf_features',numFeatures=1000)
idf = IDF(inputCol='tf_features', outputCol="tfidf_features", minDocFreq=5) #minDocFreq: remove sparse terms
# convert labels (string) to integers. Easy to process compared to string.

# Indexing the Categorical variable

telecommuting_indexer = StringIndexer(inputCol='telecommuting',outputCol='telecommuting_index',handleInvalid='keep')
has_company_logo_indexer = StringIndexer(inputCol='has_company_logo',outputCol='has_company_logo_index',handleInvalid='keep')
has_questions_indexer = StringIndexer(inputCol='has_questions',outputCol='has_questions_index',handleInvalid='keep')


# Vector assembler is used to create a vector of input features

assembler = VectorAssembler(inputCols=['tfidf_features','telecommuting_index','has_company_logo_index','has_questions_index'], outputCol="features")


lr_model = LogisticRegression(labelCol='fraudulent')#performing Logistic regression model on Fradulent categorical variable

#pipeline is used to run in a systematic way
pipeline = Pipeline(stages=[hashingtf, idf, telecommuting_indexer, has_company_logo_indexer, has_questions_indexer,assembler,lr_model])

pipelineFit_lr = pipeline.fit(train_set)

# Transforming the train set
train_data_lr = pipelineFit_lr.transform(train_set)

# Transforming the test set
test_data_lr = pipelineFit_lr.transform(test_set)

train_data_lr.show()

+-------------+----------------+-------------+----------+--------------------+--------------------+--------------------+--------------------+-------------------+----------------------+-------------------+--------------------+--------------------+--------------------+----------+
telecommuting|has_company_logo|has_questions|fraudulent| words_stemmed| words| tf_features| tfidf_features|telecommuting_index|has_company_logo_index|has_questions_index| features| rawPrediction| probability|prediction|
+-------------+----------------+-------------+----------+--------------------+--------------------+--------------------+--------------------+-------------------+----------------------+-------------------+--------------------+--------------------+--------------------+----------+
 0| 0| 0| 0|[, , servicenow, ...|[servicenow, cons...|(1000,[2,9,18,22,...|(1000,[2,9,18,22,...| 0.0| 1.0| 0.0|(1003,[2,9,18,22,...|[25.6555213057262...|[0.99999999999278...| 0.0|
 0| 0| 0| 0|[, physician, adv...|[physician, advis...|(1000,[2,10,23,45...|(1000,[2,10,23,45...| 0.0| 1.0| 0.0|(1003,[2,10,23,45...|[115.615702865910...|[1.0,6.1480617945...| 0.0|
 0| 0| 0| 0|[account, , lead,...|[account, lead, p...|(1000,[1,13,23,30...|(1000,[1,13,23,30...| 0.0| 1.0| 0.0|(1003,[1,13,23,30...|[59.3314662335184...|[1.0,1.7087227888...| 0.0|
 0| 0| 0| 0|[account, clerk, ...|[account, clerk, ...|(1000,[4,14,20,22...|(1000,[4,14,20,22...| 0.0| 1.0| 0.0|(1003,[4,14,20,22...|[443.752380499378...|[1.0,1.9089291845...| 0.0|
 0| 0| 0| 0|[account, directo...|[account, directo...|(1000,[2,22,25,36...|(1000,[2,22,25,36...| 0.0| 1.0| 0.0|(1003,[2,22,25,36...|[228.405282893386...|[1.0,6.3803720201...| 0.0|
 0| 0| 0| 0|[account, execut,...|[account, execut,...|(1000,[2,6,8,9,18...|(1000,[2,6,8,9,18...| 0.0| 1.0| 0.0|(1003,[2,6,8,9,18...|[309.134742819132...|[1.0,5.5524803346...| 0.0|
 0| 0| 0| 0|[account, manag, ...|[account, manag, ...|(1000,[23,72,101,...|(1000,[23,72,101,...| 0.0| 1.0| 0.0|(1003,[23,72,101,...|[13.5674335101684...|[0.99999871844297...| 0.0|
 0| 0| 0| 0|[account, manag, ...|[account, manag, ...|(1000,[19,23,25,7...|(1000,[19,23,25,7...| 0.0| 1.0| 0.0|(1003,[19,23,25,7...|[57.5607322286649...|[1.0,1.0039026919...| 0.0|
 0| 0| 0| 0|[account, manag, ...|[account, manag, ...|(1000,[2,12,19,22...|(1000,[2,12,19,22...| 0.0| 1.0| 0.0|(1003,[2,12,19,22...|[258.989172398246...|[1.0,3.3299027856...| 0.0|
 0| 0| 0| 0|[account, manager...|[account, manager...|(1000,[34,38,39,7...|(1000,[34,38,39,7...| 0.0| 1.0| 0.0|(1003,[34,38,39,7...|[20.4213424399697...|[0.99999999864754...| 0.0|
 0| 0| 0| 0|[accountingdata, ...|[accountingdata, ...|(1000,[3,62,63,72...|(1000,[3,62,63,72...| 0.0| 1.0| 0.0|(1003,[3,62,63,72...|[28.2658055040139...|[0.99999999999947...| 0.0|
 0| 0| 0| 0|[admin, assista, ...|[admin, assista, ...|(1000,[0,2,6,8,9,...|(1000,[0,2,6,8,9,...| 0.0| 1.0| 0.0|(1003,[0,2,6,8,9,...|[135.528830041926...|[1.0,1.3822193675...| 0.0|
 0| 0| 0| 0|[advertis, salesa...|[advertis, salesa...|(1000,[2,8,25,66,...|(1000,[2,8,25,66,...| 0.0| 1.0| 0.0|(1003,[2,8,25,66,...|[105.136881215419...|[1.0,2.1859118459...| 0.0|
 0| 0| 0| 0|[advertis, salesa...|[advertis, salesa...|(1000,[2,8,12,77,...|(1000,[2,8,12,77,...| 0.0| 1.0| 0.0|(1003,[2,8,12,77,...|[65.7792251195736...|[1.0,2.7067335539...| 0.0|
 0| 0| 0| 0|[air, environment...|[air, environment...|(1000,[4,14,22,25...|(1000,[4,14,22,25...| 0.0| 1.0| 0.0|(1003,[4,14,22,25...|[377.25735445678,...|[1.0,1.4428218213...| 0.0|
 0| 0| 0| 0|[air, qualiti, en...|[air, qualiti, en...|(1000,[4,14,22,25...|(1000,[4,14,22,25...| 0.0| 1.0| 0.0|(1003,[4,14,22,25...|[445.461195383428...|[1.0,3.4566940568...| 0.0|
 0| 0| 0| 0|[android, develop...|[android, develop...|(1000,[4,21,25,27...|(1000,[4,21,25,27...| 0.0| 1.0| 0.0|(1003,[4,21,25,27...|[294.513678530663...|[1.0,1.2426092125...| 0.0|
 0| 0| 0| 0|[android, engin, ...|[android, engin, ...|(1000,[7,9,26,38,...|(1000,[7,9,26,38,...| 0.0| 1.0| 0.0|(1003,[7,9,26,38,...|[222.916709382659...|[1.0,1.54

In [0]:
test_data_lr.select(['fraudulent','prediction']).show()

+----------+----------+
fraudulent|prediction|
+----------+----------+
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+----------+----------+
only showing top 20 rows

##Evaluating the model

##1. Area under the ROC

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
AUC_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='fraudulent',metricName='areaUnderROC')
AUC = AUC_evaluator.evaluate(test_data_lr)
print("The area under the curve is {}".format(AUC))

The area under the curve is 0.8338283251805264

##2. Area under the PR

In [0]:
PR_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='fraudulent',metricName='areaUnderPR')
PR = PR_evaluator.evaluate(test_data_lr)
print("The area under the PR curve is {}".format(PR))

The area under the PR curve is 0.5662725180939631

##3. Accuracy

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
ACC_evaluator = MulticlassClassificationEvaluator(labelCol="fraudulent", predictionCol="prediction", metricName="accuracy")
accuracy = ACC_evaluator.evaluate(test_data_lr)
print("The accuracy of the model is {}".format(accuracy))

The accuracy of the model is 0.9656585365853658

##4. Confusion Matrix

In [0]:
from sklearn.metrics import confusion_matrix
y_true = test_data_lr.select("fraudulent")
y_true = y_true.toPandas()

y_pred = test_data_lr.select("prediction")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true, y_pred)
print("Below is the confusion matrix \n {}".format(cnf_matrix))

Below is the confusion matrix 
 [[4767 93]
 [ 83 182]]

##5.F1 Score

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
f1_evaluator = MulticlassClassificationEvaluator(labelCol="fraudulent", predictionCol="prediction", metricName="f1")
f1 = f1_evaluator.evaluate(test_data_lr)
print("The F1 score of the model is {}".format(f1))

The F1 score of the model is 0.965958829543668

###Naive Bayes Model

In [0]:
df3 = df3.withColumn('fraudulent',col('fraudulent').cast('string'))

In [0]:
df3.printSchema()

root
-- telecommuting: string (nullable = false)
-- has_company_logo: string (nullable = false)
-- has_questions: string (nullable = false)
-- fraudulent: string (nullable = true)
-- text: string (nullable = false)

In [0]:
from pyspark.ml.classification import NaiveBayes

labelIndexer = StringIndexer(inputCol="fraudulent", outputCol="label")

# Train a NaiveBayes model
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",labelCol='label')

pipeline_nb = Pipeline(stages=[labelIndexer,hashingtf, idf, telecommuting_indexer, has_company_logo_indexer, has_questions_indexer,assembler,nb])

pipelineFit_nb = pipeline_nb.fit(train_set)

# Transforming the train set
test_data_nb = pipelineFit_nb.transform(test_set)


In [0]:
test_data_nb.printSchema()

root
-- telecommuting: string (nullable = false)
-- has_company_logo: string (nullable = false)
-- has_questions: string (nullable = false)
-- fraudulent: integer (nullable = true)
-- words_stemmed: array (nullable = true)
 |-- element: string (containsNull = true)
-- words: array (nullable = true)
 |-- element: string (containsNull = true)
-- label: double (nullable = false)
-- tf_features: vector (nullable = true)
-- tfidf_features: vector (nullable = true)
-- telecommuting_index: double (nullable = false)
-- has_company_logo_index: double (nullable = false)
-- has_questions_index: double (nullable = false)
-- features: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [0]:
# Select results to view
test_data_nb.select("label", "prediction", "probability").show(5)

+-----+----------+--------------------+
label|prediction| probability|
+-----+----------+--------------------+
 0.0| 0.0|[0.99966983873845...|
 0.0| 0.0|[0.99994580000875...|
 0.0| 0.0|[0.99894712442731...|
 0.0| 0.0|[0.99999999999999...|
 0.0| 0.0|[0.96198075408897...|
+-----+----------+--------------------+
only showing top 5 rows

###1.Accuracy

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(test_data_nb)
print ("Model Accuracy: ", accuracy)

Model Accuracy: 0.9166829268292683

###2.Precision

In [0]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluator.evaluate(test_data_nb)
print("Precision = %g" % (precision))
print("Test Error = %g " % (1.0 - precision))

Precision = 0.952272
Test Error = 0.0477284

###3.Recall

In [0]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="weightedRecall")
recall = evaluator.evaluate(test_data_nb)
print("Recall = %g" % (recall))
print("Test Error = %g " % (1.0 - recall))

Recall = 0.916683
Test Error = 0.0833171

##4.Confusion Matrix

In [0]:
from sklearn.metrics import confusion_matrix
y_true = test_data_nb.select("label")
y_true = y_true.toPandas()

y_pred = test_data_nb.select("prediction")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true, y_pred)
print("Below is the confusion matrix \n {}".format(cnf_matrix))

Below is the confusion matrix 
 [[4502 358]
 [ 69 196]]

##5.F1 Score

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = f1_evaluator.evaluate(test_data_nb)
print("The F1 score of the model is {}".format(f1))

The F1 score of the model is 0.9301063792233807

We have applied two classification models on the data,which are Logistic regression and Naive bayes.Using evaluation metric measures,we can see the F1 score of Logistic Regression model is better than the Naive bayes model.F1 score of the Logistic regression model is = 0.965958829543668 and for Naive bayes model is = 0.9301063792233807.

###Logistic Regression

In [0]:
train = df_final.sampleBy("fraudulent", fractions={0: 0.9, 1: 0.1}, seed=10)
test = df_final.subtract(train)

splitted the data into train and test usning samplby(strartified split),usning fractions 0.9 and 0.1

In [0]:
pipelineFit_lr_new = pipeline.fit(train)

# Transforming the test set
test_data_lr_new = pipelineFit_lr_new.transform(test)


In [0]:
test_data_lr_new.select(['fraudulent','prediction']).show()

+----------+----------+
fraudulent|prediction|
+----------+----------+
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 1| 1.0|
 0| 0.0|
 1| 0.0|
 1| 1.0|
 1| 1.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
+----------+----------+
only showing top 20 rows

###Confusion Matrix

In [0]:
from sklearn.metrics import confusion_matrix
y_true = test_data_lr_new.select("fraudulent")
y_true = y_true.toPandas()

y_pred = test_data_lr_new.select("prediction")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true, y_pred)
print("Below is the confusion matrix \n {}".format(cnf_matrix))

Below is the confusion matrix 
 [[1516 27]
 [ 368 375]]

###F1 Score

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
f1_evaluator = MulticlassClassificationEvaluator(labelCol="fraudulent", predictionCol="prediction", metricName="f1")
f1 = f1_evaluator.evaluate(test_data_lr_new)
print("The F1 score of the model is {}".format(f1))

The F1 score of the model is 0.8100757877166505

The fraudulent column count is imbalanced,so we applied startified split on the dataset.Fradulent values percentage has mentioned in the fraction field. for 0 the fraction value is 0.9 and for 1 it is 0.1. For the imbalanced data, best evaluation metric f1 score and confusion matrix.F1 score value is  = 0.8100757877166505 which is less compared to the other logistic rgerssion model value,because % of 1 values were taken very low.

###Models Saving

In [0]:
pipelineFit_nb.save('/FileStore/tables/NaivebayesModelold')
pipelineFit_lr.save('/FileStore/tables/LogisticRegressionModelold')
pipelineFit_lr_new.save('/FileStore/tables/LogisticRegressionModelnew')

##Reference
#####https://stackoverflow.com/questions/53579444/efficient-text-preprocessing-using-pyspark-clean-tokenize-stopwords-stemming
#####https://towardsdatascience.com/natural-language-processing-with-pyspark-and-spark-nlp-b5b29f8faba